In [1]:
import os
import pymysql
import pandas as pd
password = os.environ.get('MYSQL_PASSWORD')

In [2]:
password = os.environ.get('MYSQL_PASSWORD')

try:
    con = pymysql.connect(
    host='localhost',
    user='root',
    password=password,
    charset='utf8mb4'
    )
    print("Connection to the database was successful!")
except pymysql.Error as e:
    print(f"An error occurred while connecting to the database: {e}")


Connection to the database was successful!


In [3]:
# calling this function makes life easy and just need to pass the name of the database I want to make a query on
def connector (database, password=password, host='localhost', 
               user='root', charset='utf8mb4',):
    
    conn = pymysql.connect(
                        host=host,
                        user=user,
                        password=password,
                        charset=charset, 
                        database=database)
    
    return conn

In [24]:
# to get my tables' names in the datanase

query = "SELECT table_name FROM information_schema.tables WHERE table_schema='sql_store';"

# Execute the query and store the results in a Pandas DataFrame
tables = pd.read_sql_query(query, con)

# Print the list of tables
print(tables)


         TABLE_NAME
0         customers
1  order_item_notes
2       order_items
3    order_statuses
4            orders
5          products
6          shippers


# Retrieving data from a single table

# SELECT

In [8]:
con = connector ('sql_inventory')
df = pd.read_sql("SELECT * FROM products", con)
df


,product_id,name,quantity_in_stock,unit_price
0,1,Foam Dinner Plate,70,1.21
1,2,"Pork - Bacon,back Peameal",49,4.65
2,3,"Lettuce - Romaine, Heart",38,3.35
3,4,"Brocolinni - Gaylan, Chinese",90,4.53
4,5,Sauce - Ranch Dressing,94,1.63
5,6,Petit Baguette,14,2.39
6,7,Sweet Pea Sprouts,98,3.29
7,8,Island Oasis - Raspberry,26,0.74
8,9,Longan,67,2.26
9,10,Broom - Push,6,1.09


In [38]:
con = connector ('sql_store')

In [39]:
df = pd.read_sql("SELECT * \
                 FROM customers \
                 WHERE birth_date>'1990-01-01'\
                 ORDER BY last_name", 
                 con)
df


,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
1,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
2,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073


In [46]:
df = pd.read_sql("SELECT first_name, last_name \
                    FROM customers", 
                    con)
df


,first_name,last_name
0,Babara,MacCaffrey
1,Ines,Brushfield
2,Freddi,Boagey
3,Ambur,Roseburgh
4,Clemmie,Betchley
5,Elka,Twiddell
6,Ilene,Dowson
7,Thacher,Naseby
8,Romola,Rumgay
9,Levy,Mynett


In [47]:
df = pd.read_sql("SELECT last_name, first_name \
                    FROM customers", 
                    con)
df


,last_name,first_name
0,MacCaffrey,Babara
1,Brushfield,Ines
2,Boagey,Freddi
3,Roseburgh,Ambur
4,Betchley,Clemmie
5,Twiddell,Elka
6,Dowson,Ilene
7,Naseby,Thacher
8,Rumgay,Romola
9,Mynett,Levy


In [67]:
pd.read_sql("SELECT first_name, last_name, points, points*10 AS '10x POINTS' \
            FROM customers", con)

,first_name,last_name,points,10x POINTS
0,Babara,MacCaffrey,2273,22730
1,Ines,Brushfield,947,9470
2,Freddi,Boagey,2967,29670
3,Ambur,Roseburgh,457,4570
4,Clemmie,Betchley,3675,36750
5,Elka,Twiddell,3073,30730
6,Ilene,Dowson,1672,16720
7,Thacher,Naseby,205,2050
8,Romola,Rumgay,1486,14860
9,Levy,Mynett,796,7960


In [76]:
pd.read_sql("SELECT DISTINCT state FROM customers", con)

,state
0,MA
1,VA
2,CO
3,FL
4,TX
5,IL
6,TN
7,CA
8,GA


In [77]:
con = connector ('sql_inventory')

In [81]:
pd.read_sql("SELECT name, unit_price, unit_price * 1.1 AS 'new price' \
            FROM products", con)

,name,unit_price,new price
0,Foam Dinner Plate,1.21,1.331
1,"Pork - Bacon,back Peameal",4.65,5.115
2,"Lettuce - Romaine, Heart",3.35,3.685
3,"Brocolinni - Gaylan, Chinese",4.53,4.983
4,Sauce - Ranch Dressing,1.63,1.793
5,Petit Baguette,2.39,2.629
6,Sweet Pea Sprouts,3.29,3.619
7,Island Oasis - Raspberry,0.74,0.814
8,Longan,2.26,2.486
9,Broom - Push,1.09,1.199


In [82]:
con = connector('sql_store')

# WHERE

In [83]:
pd.read_sql("SELECT * FROM customers WHERE points>3000", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
1,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073


In [87]:
pd.read_sql('SELECT * FROM customers WHERE state != "VA"', con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
2,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
3,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
4,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
5,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
6,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
7,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
8,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [89]:
pd.read_sql("SELECT * FROM customers WHERE birth_date > '1990-01-01'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
1,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
2,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


In [3]:
# to get my tables' names in the datanase

query = "SELECT table_name FROM information_schema.tables WHERE table_schema='sql_store';"

# Execute the query and store the results in a Pandas DataFrame
tables = pd.read_sql_query(query, con)

# Print the list of tables
print(tables)


         TABLE_NAME
0         customers
1  order_item_notes
2       order_items
3    order_statuses
4            orders
5          products
6          shippers


In [98]:
pd.read_sql("SELECT * FROM orders WHERE order_date>='2019-01-01'", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,1,6,2019-01-30,1,None,None,None


In [6]:
con = connector('sql_store')

# AND OR NOT

In [13]:
pd.read_sql("SELECT * \
            FROM customers \
            WHERE birth_date > '1990-01-01' AND points>1000", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
1,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


In [14]:
pd.read_sql("SELECT * \
            FROM customers \
            WHERE birth_date > '1990-01-01' OR points>1000", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
2,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
3,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
4,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
5,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
6,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


In [17]:
pd.read_sql("SELECT * \
            FROM customers \
            WHERE birth_date > '1990-01-01' OR \
            (points>1000 AND state ='VA')", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
1,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
2,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


In [21]:
pd.read_sql("SELECT * \
            FROM customers \
            WHERE NOT (birth_date > '1990-01-01' OR points>1000)", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
1,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
2,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [23]:
pd.read_sql("SELECT * \
            FROM customers \
            WHERE  birth_date <= '1990-01-01' AND points<=1000", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
1,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
2,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [31]:
con = connector('sql_store')

pd.read_sql("SELECT * \
            FROM order_items \
            WHERE (order_id = 6 AND (unit_price * quantity)> 30)", con)

,order_id,product_id,quantity,unit_price
0,6,1,4,8.65


In [34]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE state ='VA' OR state = 'GA' OR state = 'FL'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
1,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
2,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
3,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


# IN

In [35]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE state IN ('VA', 'FL','GA')", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
1,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
2,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
3,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [36]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE state NOT IN ('VA', 'FL','GA')", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
2,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
3,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
4,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
5,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


In [39]:
pd.read_sql("SELECT *\
            FROM products\
            WHERE quantity_in_stock IN (49, 38, 72)", con)

,product_id,name,quantity_in_stock,unit_price
0,2,"Pork - Bacon,back Peameal",49,4.65
1,3,"Lettuce - Romaine, Heart",38,3.35


In [40]:
pd.read_sql("SELECT *\
            FROM customers\
                WHERE points >=1000 AND points <=3000", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
2,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
3,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


# BETWEEN

In [43]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE points BETWEEN 1000 AND 3000", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
2,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
3,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


In [48]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE birth_date BETWEEN '1990-01-01' AND '2000-01-01'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
1,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
2,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


# LIKE

In [49]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name LIKE 'b%'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
1,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
2,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675


In [50]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name LIKE 'brush%'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947


In [51]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name LIKE '%b%'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
1,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
2,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
3,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
4,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205


In [52]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name LIKE '%y'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
2,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
3,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
4,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


In [53]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name LIKE '_y'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points


In [55]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name LIKE '_____y'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
1,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
2,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


In [56]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name LIKE 'b____y'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967


In [69]:
pd.read_sql("SELECT*\
            FROM customers\
            WHERE address LIKE '%TRAIL%' OR \
                  address LIKE '%AVENUE%'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
1,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
2,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [70]:
pd.read_sql("SELECT*\
            FROM customers\
            WHERE phone LIKE '%9'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
1,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672


In [71]:
pd.read_sql("SELECT*\
            FROM customers\
            WHERE phone NOT LIKE '%9'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
3,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
4,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
5,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
6,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [4]:
con = connector('sql_store')

In [5]:
pd.read_sql('SELECT * FROM customers', con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
3,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
4,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
5,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
6,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
7,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
8,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
9,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


# REGEXP

In [6]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name LIKE '%field%'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947


In [7]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP 'field'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947


In [8]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP '^Brush'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947


In [9]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP 'field$'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947


In [14]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP 'field|mac'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947


In [15]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP 'field|mac|rose'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457


In [16]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP '^field|mac|rose'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457


In [17]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP 'field$|mac|rose'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457


In [23]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP '[gim]e'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
1,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967


In [25]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP 'e[l]'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
1,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073


In [26]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP '[a-h]e'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
1,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
2,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073


In [27]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE first_name REGEXP 'ELKA|AMBUR'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
1,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073


In [28]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP 'EY$|ON$'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
2,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
3,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672


In [29]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE last_name REGEXP '^MY|SE'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
1,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
2,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [31]:
pd.read_sql("SELECT * \
            FROM customers\
            WHERE last_name REGEXP 'B[RU]'", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
1,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457


# IS NULL

In [35]:
pd.read_sql("SELECT * \
            FROM customers", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
3,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
4,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
5,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
6,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
7,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
8,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
9,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [38]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE phone IS NULL", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675


In [39]:
pd.read_sql("SELECT *\
            FROM customers\
            WHERE phone IS NOT NULL", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
3,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
4,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
5,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
6,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
7,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
8,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [40]:
# to get my tables' names in the datanase

query = "SELECT table_name FROM information_schema.tables WHERE table_schema='sql_store';"

# Execute the query and store the results in a Pandas DataFrame
tables = pd.read_sql_query(query, con)

# Print the list of tables
print(tables)


         TABLE_NAME
0         customers
1  order_item_notes
2       order_items
3    order_statuses
4            orders
5          products
6          shippers


In [48]:
pd.read_sql("SELECT *\
            FROM orders", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,1,6,2019-01-30,1,None,None,NaN
1,2,7,2018-08-02,2,None,2018-08-03,4.0
2,3,8,2017-12-01,1,None,None,NaN
3,4,2,2017-01-22,1,None,None,NaN
4,5,5,2017-08-25,2,,2017-08-26,3.0
5,6,10,2018-11-18,1,Aliquam erat volutpat. In congue.,None,NaN
6,7,2,2018-09-22,2,None,2018-09-23,4.0
7,8,5,2018-06-08,1,"Mauris enim leo, rhoncus sed, vestibulum sit a...",None,NaN
8,9,10,2017-07-05,2,Nulla mollis molestie lorem. Quisque ut erat.,2017-07-06,1.0
9,10,6,2018-04-22,2,None,2018-04-23,2.0


In [52]:
pd.read_sql("SELECT *\
            FROM orders\
            WHERE shipped_date IS NULL", con)

,order_id,customer_id,order_date,status,comments,shipped_date,shipper_id
0,1,6,2019-01-30,1,None,None,None
1,3,8,2017-12-01,1,None,None,None
2,4,2,2017-01-22,1,None,None,None
3,6,10,2018-11-18,1,Aliquam erat volutpat. In congue.,None,None
4,8,5,2018-06-08,1,"Mauris enim leo, rhoncus sed, vestibulum sit a...",None,None


# ORDER BY

In [53]:
pd.read_sql("SELECT *\
            FROM customers\
            ORDER BY first_name", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
1,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
2,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
3,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
4,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
5,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
6,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
7,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796
8,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
9,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205


In [54]:
pd.read_sql("SELECT *\
            FROM customers\
            ORDER BY first_name DESC", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
1,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
2,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796
3,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
4,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
5,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
6,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
7,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
8,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
9,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457


In [55]:
pd.read_sql("SELECT * \
            FROM customers\
            ORDER BY state, first_name", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
1,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
2,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
3,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
4,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796
5,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
6,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
7,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
8,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
9,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947


In [56]:
pd.read_sql("SELECT * \
            FROM customers\
            ORDER BY state DESC, first_name", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
1,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
2,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
3,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
4,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
5,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796
6,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
7,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
8,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
9,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


In [58]:
pd.read_sql("SELECT first_name, last_name\
            FROM customers\
            ORDER BY birth_date", con)

,first_name,last_name
0,Ilene,Dowson
1,Levy,Mynett
2,Clemmie,Betchley
3,Ambur,Roseburgh
4,Freddi,Boagey
5,Babara,MacCaffrey
6,Ines,Brushfield
7,Elka,Twiddell
8,Romola,Rumgay
9,Thacher,Naseby


In [60]:
pd.read_sql("SELECT first_name, last_name, 10 AS points\
            FROM customers\
            ORDER BY points, birth_date", con)

,first_name,last_name,points
0,Ilene,Dowson,10
1,Levy,Mynett,10
2,Clemmie,Betchley,10
3,Ambur,Roseburgh,10
4,Freddi,Boagey,10
5,Babara,MacCaffrey,10
6,Ines,Brushfield,10
7,Elka,Twiddell,10
8,Romola,Rumgay,10
9,Thacher,Naseby,10


In [72]:
pd.read_sql("SELECT *, quantity * unit_price AS total_price\
            FROM order_items\
            WHERE order_id = 2\
            ORDER BY total_price DESC", con)

,order_id,product_id,quantity,unit_price,total_price
0,2,1,2,9.10,18.20
1,2,4,4,1.66,6.64
2,2,6,2,2.94,5.88


# LIMIT

In [75]:
pd.read_sql("SELECT *\
            FROM customers\
            LIMIT 4", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
3,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457


In [76]:
pd.read_sql("SELECT *\
            FROM customers\
            LIMIT 400", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,1,Babara,MacCaffrey,1986-03-28,781-932-9754,0 Sage Terrace,Waltham,MA,2273
1,2,Ines,Brushfield,1986-04-13,804-427-9456,14187 Commercial Trail,Hampton,VA,947
2,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967
3,4,Ambur,Roseburgh,1974-04-14,407-231-8017,30 Arapahoe Terrace,Orlando,FL,457
4,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
5,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
6,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
7,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
8,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486
9,10,Levy,Mynett,1969-10-13,404-246-3370,68 Lawn Avenue,Atlanta,GA,796


In [79]:
pd.read_sql("SELECT *\
            FROM customers\
            LIMIT 6, 3", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,7,Ilene,Dowson,1964-08-30,615-641-4759,50 Lillian Crossing,Nashville,TN,1672
1,8,Thacher,Naseby,1993-07-17,941-527-3977,538 Mosinee Center,Sarasota,FL,205
2,9,Romola,Rumgay,1992-05-23,559-181-3744,3520 Ohio Trail,Visalia,CA,1486


In [82]:
pd.read_sql("SELECT *\
            FROM customers\
            ORDER BY points DESC\
            LIMIT 3", con)

,customer_id,first_name,last_name,birth_date,phone,address,city,state,points
0,5,Clemmie,Betchley,1973-11-07,None,5 Spohn Circle,Arlington,TX,3675
1,6,Elka,Twiddell,1991-09-04,312-480-8498,7 Manley Drive,Chicago,IL,3073
2,3,Freddi,Boagey,1985-02-07,719-724-7869,251 Springs Junction,Colorado Springs,CO,2967


# Retrieving data from multiple tables